In [220]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import networkx as nx
from IPython.display import Audio, display
import ipynb.fs.defs.Utils as Utils

In [221]:
def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/001/131/original/kon.wav', autoplay=True))

In [222]:
#This class contains the methods and properties related to the features of an image
class ImageFeature:
    def __init__(self, image, index):
        self.image = image
        self.index = index  
        
    #Compute the salient points of the image using SIFT algorithm
    def SIFT(self, save_output = False, output_dir = "output"):
        sift = cv.SIFT_create()
        self.kp, self.des = sift.detectAndCompute(self.image,None)
        self.img_with_sift=cv.drawKeypoints(self.image,self.kp,self.image)
        if save_output:
            cv.imwrite(os.path.join(output_dir,f"{self.index+1}.jpg"), self.img_with_sift)

In [223]:
#This class contains the methods and properties related to the problem of feature matching between two images
class Match:
    def __init__(self, image_feature_source, image_feature_destination):
        self.image_feature_source = image_feature_source
        self.image_feature_destination = image_feature_destination
    
    #This function performs feature matching between to images
    def feature_matching(self, threshold, save_output = False, output_dir = "output"):
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(self.image_feature_source.des,
                                 self.image_feature_destination.des,
                                 k=2)
        # Need to draw only good matches, so create a mask
        self.good = []
        for m,n in matches:
            if m.distance < threshold*n.distance:
                self.good.append((m,n))
        draw_params = dict(matchColor = (0,255,0),
                           singlePointColor = (255,0,0),
                           flags = cv.DrawMatchesFlags_DEFAULT)
        self.img_with_match = cv.drawMatchesKnn(self.image_feature_source.image,
                                                self.image_feature_source.kp,
                                                self.image_feature_destination.image,
                                                self.image_feature_destination.kp,
                                                self.good,None,**draw_params)
        if save_output:
            cv.imwrite(os.path.join(output_dir,f"match_{self.image_feature_source.index+1}_{ self.image_feature_destination.index+1}.jpg"),self.img_with_match)
    
    #This function allows to check whether the match between the source and destination images is to be considered a good match
    def check_salient(self, threshold, save_output = False, output_dir = "output"):
        #A threshold is used to define the goodness of the  match
        if len(self.good) > threshold:
            if save_output:
                cv.imwrite(os.path.join(output_dir,f"salient_{self.image_feature_source.index+1}_{ self.image_feature_destination.index+1}.jpg"),self.img_with_match)
            return True
        return False
    
    def mult_and_norm(self,H, points):
        proj_p = np.dot(H,points)
        proj_p = proj_p / proj_p[2,:]
        return proj_p[0:2,:]
    
    #This function allows to estimate the homography between the two images
    def fit_homography(self, RANSACmaxIters = 2000, T_norm = np.eye(3)):
        self.src_pts = np.float32([self.image_feature_source.kp[m[0].queryIdx].pt for m in self.good])
        self.dst_pts = np.float32([self.image_feature_destination.kp[m[0].trainIdx].pt for m in self.good])
        
        self.src_pts = np.concatenate((self.src_pts, np.ones([self.src_pts.shape[0],1])), axis=1)
        self.dst_pts = np.concatenate((self.dst_pts, np.ones([self.dst_pts.shape[0],1])), axis=1)
        
        
        src_pts_proj = self.mult_and_norm(np.eye(3),self.src_pts.transpose()).transpose()
        dst_pts_proj = self.mult_and_norm(np.eye(3),self.dst_pts.transpose()).transpose()
        
        
        self.M, self.mask = cv.findHomography(src_pts_proj, dst_pts_proj, cv.RANSAC, 3.0, maxIters=RANSACmaxIters)
        self.M = T_norm @ self.M @ np.linalg.inv(T_norm)
    
    #This function allows to normalize the homography so that the determinant is unitary
    #This makes homographies part of the SL(3) group
    def normalize_homography(self):
        det = np.linalg.det(self.M)
        self.H = self.M/np.cbrt(det)
        new_det = np.linalg.det(self.H)
        
    #Remember: if src is image I and dest is image J, we are finding H from I to J, this is 
    #Hj,i
    

In [224]:
def compute_matches(imgs,
                    T_norm,
                   save_images = False,
                   matching_threshold = 0.6,
                   matches_dir = "matches",
                   matches_th = 20,
                   number_of_matches=1,
                   salient_matches_dir = "salient_matches",
                   sift_dir = "sift",
                   RANSACmaxIters = 2000):
    n = len(imgs)
    image_features =[] #Array that will contain the salient features of each image
    for i in range(0,n):
        image_feature = ImageFeature(imgs[i], i)
        image_feature.SIFT(save_images, sift_dir)
        image_features.append(image_feature)
        
    #Compute the good matches between each pair of images
    matches =[]
    for i in range(0, n-1):
        for j in range(i+1, n):
            for k in range(0,number_of_matches):
                match = Match(image_features[i], image_features[j])
                match.feature_matching(matching_threshold, save_images, matches_dir)
                matches.append(match)


    salient_matches = list(filter(lambda x: x.check_salient(matches_th, save_images, salient_matches_dir),matches))
    
    #For every good match, compute also the matches in the reverse order (destination, source) and compute homographies
    total_matches = []
    for match in salient_matches:
        inv_match = Match(match.image_feature_destination, match.image_feature_source)
        inv_match.feature_matching(matching_threshold, save_images, matches_dir)
        match.fit_homography(RANSACmaxIters, T_norm)
        match.normalize_homography()
        inv_match.fit_homography(RANSACmaxIters, T_norm)
        inv_match.normalize_homography()
        total_matches.append(match)
        total_matches.append(inv_match)
        
    return total_matches

In [225]:
def expand_graph(adj_matrix):
    replicas_structure = dict()
    adj_matrix_exp = np.copy(adj_matrix)
    
    #more multiedges, we have to expand
    while np.sum(adj_matrix_exp>1)>0:
        
        adj_matrix_multi = np.maximum(adj_matrix_exp-1,0)
        edges_out = np.sum(adj_matrix_multi, axis=1)
        node_max = np.argmax(edges_out)
        replicas = np.max(adj_matrix_multi[node_max,:])
        appended_column = np.copy(adj_matrix_exp[:,node_max])
        appended_column[node_max]=1
        for i in range(replicas):
            adj_matrix_exp = np.c_[adj_matrix_exp,appended_column]

        row=np.copy(adj_matrix_exp[node_max,:])

        appended_row = np.minimum(1,row)
        adj_matrix_exp[node_max,:] = appended_row
        row = np.maximum(0,row-1)
        for i in range(replicas):
            appended_row = np.minimum(1,row)
            adj_matrix_exp = np.r_[adj_matrix_exp, [appended_row]]
            row = np.maximum(0,row-1)
        
        replicas_structure[node_max] = [node_max] +  list(range(adj_matrix_exp.shape[0]-replicas, adj_matrix_exp.shape[0]))
        
    return adj_matrix_exp, replicas_structure

In [226]:
def compute_Z_matrix(n, replicas, matches):

    matches_handled = np.zeros([n,n], dtype=int)
    Z = np.eye(3*n, 3*n)
    for match in matches:
        
        i_temp=match.image_feature_source.index
        j=match.image_feature_destination.index
        if i_temp in replicas:
            i = replicas[i_temp][(matches_handled[i_temp,j])]
        else:
            i=i_temp    
        if j in replicas:
            for k in replicas[j]:
                Z[3*k:3*(k+1),3*i:3*(i+1)] = match.H
        else:
            Z[3*j:3*(j+1),3*i:3*(i+1)] = match.H
        matches_handled[i_temp,j]+=1

    for replica in replicas:
        r = replicas[replica]
        for k in r:
            #add identity constraint (replica -> original node)
            Z[3*k:3*(k+1),3*r[0]:3*(r[0]+1)] = np.eye(3)
    return Z

In [227]:
def compute_C_matrix(n_expanded, n, replicas):
    C = np.zeros([3*n_expanded, 3*(n_expanded-n)],dtype=int)
    for replica in replicas:
        r = replicas[replica]
        first = r[0]
        for k in r:
            if k != first:
                C[3*first:3*(first+1),3*(k-n):3*(k-n+1)] = np.eye(3)
                C[3*k:3*(k+1),3*(k-n):3*(k-n+1)] = -np.eye(3)
    return C

In [228]:
def compute_M_matrix(adj_matrix, n, Z):
    D = np.diag(np.sum(adj_matrix + np.eye(n), axis=1))
    D_kron = np.kron(D, np.eye(3))
    M = Z - D_kron
    return M

In [229]:
def build_graph_matrices(dataset_name,
                imgs,
                T_norm,
                matching_threshold = 0.6,
                number_of_matches = 1,
                matches_th = 20,
                RANSACmaxIters = 2000,
                save_output = True,
                save_images = True,
                output_dir ="output",
                results_dir = "results",
                verbose = True
               ):
    
        M_filename = f"M_{dataset_name}.npy"
        C_filename = f"C_{dataset_name}.npy"
        Z_filename = f"Z_{dataset_name}.npy"
        Adj_filename = f"Adj_{dataset_name}.npy"
        Weight_filename = f"W_{dataset_name}.npy"
        
        sift_dir = os.path.join(results_dir,'sift')
        matches_dir = os.path.join(results_dir,'matches')
        salient_matches_dir = os.path.join(results_dir,'salient_matches')
        graph_dir = os.path.join(results_dir,'graph')
        
        if save_images:
            shutil.rmtree(results_dir)
            if not os.path.isdir(results_dir):   
                os.mkdir(results_dir)
            if not os.path.isdir(sift_dir):
                os.mkdir(sift_dir)
            if not os.path.isdir(matches_dir):
                os.mkdir(matches_dir)
            if not os.path.isdir(salient_matches_dir):
                os.mkdir(salient_matches_dir)   
            if not os.path.isdir(graph_dir):
                os.mkdir(graph_dir)
        
        if save_output:
            if not os.path.isdir(output_dir):
                os.mkdir(output_dir)
            
        n = len(imgs)
        
        matches = compute_matches(imgs = imgs, 
                                  T_norm = T_norm,
                                  save_images= save_images, 
                                  matching_threshold = matching_threshold, 
                                  matches_dir = matches_dir, 
                                  number_of_matches = number_of_matches,
                                  salient_matches_dir = salient_matches_dir, 
                                  sift_dir = sift_dir,
                                  RANSACmaxIters = RANSACmaxIters)
        
        #compute the adj and weight matrix of matches
        weight_matrix = np.zeros([n,n])
        adj_matrix = np.zeros([n,n], dtype=int)
        for match in matches:
            adj_matrix[match.image_feature_source.index, match.image_feature_destination.index]+=1
            weight_matrix[match.image_feature_source.index, match.image_feature_destination.index]=len(match.good)
            
        adj_matrix_exp, replicas = expand_graph(adj_matrix)
        
        
        if verbose:       
            Utils.build_and_print_multi_graph(adj_matrix, save_images, graph_dir, "graph")       #not expanded
            Utils.build_and_print_multi_graph(adj_matrix_exp, save_images, graph_dir, "expanded_graph")   #expanded
        
        n_expanded = adj_matrix_exp.shape[0]
        Z = compute_Z_matrix(n_expanded, replicas, matches)
        
        C = compute_C_matrix(n_expanded, n, replicas)
        
        M = compute_M_matrix(adj_matrix_exp, n_expanded, Z)
        
        if save_output:
            np.save(os.path.join(output_dir,M_filename), M)
            np.save(os.path.join(output_dir,Z_filename), Z)
            np.save(os.path.join(output_dir,C_filename), C)
            np.save(os.path.join(output_dir,Adj_filename), adj_matrix_exp)
            np.save(os.path.join(output_dir,Weight_filename), weight_matrix)
        
        if verbose:#pay attention
            allDone()
        
        return M, Z, C, adj_matrix_exp, weight_matrix
        